<a href="https://colab.research.google.com/github/zubair9703/Google-Earth-Engine-Random-Forest/blob/main/RF_for_NIR_RED_GREEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 187 kB 54.1 MB/s 
     |████████████████████████████████| 1.2 MB 39.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.0 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 8.8 MB 38.4 MB/s 
     |████████████████████████████████| 1.3 MB 47.9 MB/s 
     |████████████████████████████████| 98 kB 8.1 MB/s 
     |████████████████████████████████| 95 kB 3.7 MB/s 
     |████████████████████████████████| 3.0 MB 41.5 MB/s 
     |████████████████████████████████| 130 kB 62.2 MB/s 
     |████████████████████████████████| 99 kB 9.4 MB/s 
     |████████████████████████████████| 128 kB 58.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 344 kB 58.9 MB/s 
     |████████████████████████████████| 9.

In [2]:
import ee
import geemap


In [3]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [4]:
s2=ee.ImageCollection("COPERNICUS/S2_SR")
roi=ee.Geometry.Polygon([[[77.45036134046319, 13.177235646600929],[77.45036134046319, 13.081444893977725],[77.5583363465667, 13.081444893977725],[77.5583363465667, 13.177235646600929]]])
filtered= s2.filter(ee.Filter.date('2022-01-01','2022-04-25')).filter(ee.Filter.bounds(roi)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', .5)).first().clip(roi)

In [5]:
Map.setCenter(77.49525079053643,13.12339059592214,12)
Map.addLayer(filtered,{"bands":['B8','B4','B3'],"min":51,"max":4626},'Fcc')
Map

Map(center=[13.12339059592214, 77.49525079053643], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:
gcp=ee.FeatureCollection("projects/ee-zubahme97/assets/finaltraning_set")
bands= ['B8','B4','B3']
sample= filtered.select(bands).sampleRegions(**{"collection":gcp,"properties":['landcover'],"scale":10})
sample = sample.randomColumn()
trainingGcp = sample.filter(ee.Filter.lt('random', 0.7));
validationGcp = sample.filter(ee.Filter.gte('random', 0.7));


In [7]:
classifier = ee.Classifier.smileRandomForest(100).train(**{"features": trainingGcp,"classProperty": 'landcover',"inputProperties": bands})

In [8]:
classified= filtered.classify(classifier)

In [9]:
print(classifier.explain())

ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Classifier.explain",
    "arguments": {
      "classifier": {
        "functionInvocationValue": {
          "functionName": "Classifier.train",
          "arguments": {
            "classProperty": {
              "constantValue": "landcover"
            },
            "classifier": {
              "functionInvocationValue": {
                "functionName": "Classifier.smileRandomForest",
                "arguments": {
                  "numberOfTrees": {
                    "constantValue": 100
                  }
                }
              }
            },
            "features": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.randomColumn",
                      "arguments": {
                      

In [18]:
Map.addLayer(classified, {"min": 0, "max": 10, "palette": ['c81f04',  '100eff', '179750','ad7b0d','004902','ebff3f','ee6aff',"aaff5a","a6adba",'ba81a6','61ff4e']}, '2022')
legend_dict = {
    '0)Built-up': 'c81f04',
    '1)water': '100eff',
    '2)otherplantations': '179750',
    '3)coconut': 'ad7b0d',
    '4)mango': '004902',
    '5)openland': 'ebff3f',
    '6)fallowlands': 'ee6aff',
    '7)cropland': 'aaff5a',
    '8)scrublands': 'a6adba',
    '9)Eucalyptus': 'ba81a6',
    '10)Forest':'61ff4e'
}
# add_north_arrow(ax, text='N', xy=(0.1, 0.1), arrow_length=0.1, text_color='black', arrow_color='black', fontsize=20, width=5, headwidth=15, ha='center', va='center')
Map.add_legend(title="Classification of RE3,SWIR1 and SWIR2", legend_dict=legend_dict,position="bottomright")

In [11]:
train_accuracy = classifier.confusionMatrix()
train_accuracy.getInfo()
train_accuracy.accuracy().getInfo()
train_accuracy.kappa().getInfo()
train_accuracy.producersAccuracy().getInfo()
train_accuracy.consumersAccuracy().getInfo()

[[1,
  1,
  0.9926739926739927,
  0.996078431372549,
  0.985,
  1,
  0.9858156028368794,
  1,
  0.9954337899543378,
  0.9841628959276018,
  0.9803921568627451]]

In [12]:
validated = validationGcp.classify(classifier)

In [13]:
testConfusionMatrix = validated.errorMatrix('landcover', 'classification')
print('Confusion Matrix')
testConfusionMatrix.getInfo()

Confusion Matrix


[[93, 0, 0, 1, 0, 0, 4, 0, 1, 0, 0],
 [0, 346, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 89, 7, 3, 0, 0, 2, 2, 6, 5],
 [0, 2, 1, 94, 5, 0, 0, 0, 1, 4, 1],
 [0, 0, 0, 1, 147, 0, 2, 1, 19, 12, 0],
 [0, 0, 0, 0, 0, 15, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 49, 0, 1, 0, 0],
 [0, 0, 5, 0, 0, 0, 1, 63, 0, 0, 0],
 [0, 0, 0, 2, 43, 0, 3, 2, 54, 2, 0],
 [0, 0, 11, 3, 35, 0, 0, 0, 7, 167, 0],
 [0, 0, 29, 6, 0, 0, 0, 0, 0, 0, 14]]

In [14]:
print('Test Accuracy')
testConfusionMatrix.accuracy().getInfo()

Test Accuracy


0.8285714285714286

In [15]:
print('kappa')
testConfusionMatrix.kappa().getInfo()

kappa


0.8005971908432403

In [16]:
print('ProdcuerAccuracy')
testConfusionMatrix.producersAccuracy().getInfo()

ProdcuerAccuracy


[[0.9393939393939394],
 [0.9942528735632183],
 [0.7807017543859649],
 [0.8703703703703703],
 [0.8076923076923077],
 [0.9375],
 [0.9607843137254902],
 [0.9130434782608695],
 [0.5094339622641509],
 [0.7488789237668162],
 [0.2857142857142857]]

In [17]:
print('UserAccuracy')
testConfusionMatrix.consumersAccuracy().getInfo()

UserAccuracy


[[1,
  0.9942528735632183,
  0.6592592592592592,
  0.8103448275862069,
  0.6282051282051282,
  1,
  0.8166666666666667,
  0.9264705882352942,
  0.6352941176470588,
  0.8743455497382199,
  0.7]]

In [ ]:
import os
import csv

In [ ]:
os.chdir("/content/drive/MyDrive/ee")
home="/content/drive/MyDrive/ee"

In [ ]:

out_dir = os.path.join(home, 'Downloads')
# training_csv = os.path.join(out_dir, 'train_accuracyfcc1.csv')
testing_csv = os.path.join(out_dir, 'finalfcc1.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testConfusionMatrix.getInfo())

In [ ]:
out_dir = os.path.join(home, 'Downloads')
training_csv = os.path.join(out_dir, 'pafcc1.csv')
testing_csv = os.path.join(out_dir, 'uafcc1.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testConfusionMatrix.producersAccuracy().getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testConfusionMatrix.consumersAccuracy().getInfo())